In [ ]:
%%time
import pandas as pd
import numpy as np
import gc
import os
import random
import pickle
from sklearn.model_selection import StratifiedKFold,KFold
from scipy.sparse import hstack,vstack,csr_matrix,save_npz,load_npz
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb

############################################################################
#----- work folder -----
############################################################################
settings = json.load(open('./settings.json'))

input_path = settings['input_path']
features_path = settings['features_path']
model_path = settings['model_path']
sub_path = settings['sub_path']

# transform target by tsvd

In [ ]:
train_multi_targets = pd.read_hdf(input_path+'train_multi_targets.h5')
tsvd = TruncatedSVD(n_components=1000)
data_reduced = tsvd.fit_transform(train_multi_targets.values)

# load sparse matrix svd

In [ ]:
%%time

train_df = pd.read_feather(feature_path+'train_multi_inputs_id.feather')
test_df = pd.read_feather(feature_path+'test_multi_inputs_id.feather')
multi_inputs_svd = np.load(feature_path+'multi_inputs_svd_100.npy')
train_multi_X = multi_inputs_svd[:len(train_df)]
test_multi_X = multi_inputs_svd[len(train_df):]
train_multi_y = data_reduced  

In [ ]:
%%time

# ====================================================
# lightgbm
# ====================================================
def lgb_kfold(train_df, test_df, train_multi_X, train_multi_y, test_multi_X, folds):
    params = {    
        'objective' : 'rmse',
        'metric' : 'rmse', 
         'num_leaves': 33,
         'min_data_in_leaf': 30,
         'learning_rate': 0.02,
         'max_depth': 6,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9,
         "bagging_seed": 42,
         "verbosity": -1,        
                 }      
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    cv_corr = []
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df)): 
        print ('n_fold:',n_fold)
        train_x = train_multi_X[train_idx]
        valid_x = train_multi_X[valid_idx]
        train_y = train_multi_y[train_idx]
        valid_y = train_multi_y[valid_idx]

        dtrain = lgb.Dataset(
            train_x, label=train_y,)
        dval = lgb.Dataset(
            valid_x, label=valid_y, reference=dtrain,)
        bst = lgb.train(
            params, dtrain, num_boost_round=100000,
            valid_sets=[dval],verbose_eval=1000, early_stopping_rounds=100,
        )

        oof_preds[valid_idx] = bst.predict(valid_x, num_iteration=bst.best_iteration)
        sub_preds += bst.predict(test_multi_X, num_iteration=bst.best_iteration) / folds.n_splits         
        
    return oof_preds,sub_preds

In [ ]:
%%time
seed = 666
folds = KFold(n_splits= 5, shuffle=True, random_state=seed)  
train_preds = []
test_preds = []
for i in range(1000):
    print('=====================')
    print(i)
    train_multi_y_single = train_multi_y[:,i]
    oof_preds,sub_preds = lgb_kfold(train_df, test_df, train_multi_X, train_multi_y_single, test_multi_X, folds)
    train_preds.append(oof_preds)
    test_preds.append(sub_preds)

In [ ]:
def correlation_score(y_true, y_pred):
    """Scores the predictions according to the competition rules. 
    
    It is assumed that the predictions are not constant.
    
    Returns the average of each sample's Pearson correlation coefficient"""
    if type(y_true) == pd.DataFrame: y_true = y_true.values
    if type(y_pred) == pd.DataFrame: y_pred = y_pred.values
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)

oof_preds = np.zeros((train_multi_X.shape[0], 1000))
for n in range(len(train_preds)):
    oof_preds[:,n] =  train_preds[n]

sub_preds = np.zeros((test_multi_X.shape[0], 1000))
for n in range(len(test_preds)):
    sub_preds[:,n] =  test_preds[n]  

oof_preds_original = tsvd.inverse_transform(oof_preds)
sub_preds_original = tsvd.inverse_transform(sub_preds)    

In [ ]:
lgb = np.concatenate([oof_preds_original,sub_preds_original],axis=0)

tsvd = TruncatedSVD(n_components=100, algorithm='arpack')
lgb_svd = tsvd.fit_transform(lgb)
np.save(feature_path+'multi_lgb_svd_100.npy', lgb4_svd)